In [1]:
import sqlite3
import warnings

import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split



## Introduction


### Data Scraping
To get MLB data, I needed a web scraper.  This is because I wanted to get data at the game log level, where I had a record of each players performance after each game.  This kind of data was not readily available, so I had to scrape it.  I wrote a simple python routine which can be found under the src/ directory.  This script built an sqlite database which I could then read from

### Data Cleaning
Unfortunately after scraping several years worth of data, I noticed that the tables on the website were often wrong - the win-lose and streak columns were inverted.  Since I no longer trusted the website for these "calculated" stats, I opted to instead calculate them myself using SQL.  I include some sample corrections below but have omitted some of the work for brevity.

Finally, I updated the whole database with a few LAG statements so that for each row the wins, games played, and streak columns indicated the values exclusive of the result of the current row's game.  This will make my life a lot easier when it comes to querying data for training.
NULL values for the first game of the season were set to 0.

# Modeling part 1 - A simple linear model
To begin with, we need some sort of baseline.  I decided on a basic logistic regression model which takes only a few inputs related to the performance of the two teams facing off and attempts to predict the winner.  

I decided that we ought to feed the model the average performance of the previous five games, along with any win streak and the total win/loss percentage for the season.  The code is below.

In [2]:

# SQL Query for this data
query = """
SELECT 
    teams.year,
	teams.date,
	teams.team,
	teams.opponent,
	(SUM(teams.r) OVER (
		PARTITION BY teams.team, teams.year
		ORDER BY teams.date
		ROWS BETWEEN 5 PRECEDING AND CURRENT ROW
		) - teams.r)*1.0/5 as last_five_run,
	(SUM(teams.h) OVER (
		PARTITION BY teams.team, teams.year
		ORDER BY teams.date
		ROWS BETWEEN 5 PRECEDING AND CURRENT ROW
		) - teams.h)*1.0/5 as last_five_hit,
	(SUM(teams.e) OVER (
		PARTITION BY teams.team, teams.year
		ORDER BY teams.date
		ROWS BETWEEN 5 PRECEDING AND CURRENT ROW
		) - teams.e)*1.0/5 as last_five_err,
    CASE
        WHEN teams.games_played > 0 THEN teams.wins*1.0/(teams.games_played)
        ELSE 0.0
    END as win_pct,
	teams.streak,
    teams."lg rk" as team_rank,
	(SUM(opp.r) OVER (
		PARTITION BY opp.team, opp.year
		ORDER BY opp.date
		ROWS BETWEEN 5 PRECEDING AND CURRENT ROW
		) - opp.r)*1.0/5 as last_five_run_opp,
	(SUM(opp.h) OVER (
		PARTITION BY opp.team, opp.year
		ORDER BY opp.date
		ROWS BETWEEN 5 PRECEDING AND CURRENT ROW
		) - opp.h)*1.0/5 as last_five_hit_opp,
	(SUM(opp.e) OVER (
		PARTITION BY opp.team, opp.year
		ORDER BY opp.date
		ROWS BETWEEN 5 PRECEDING AND CURRENT ROW
		) - opp.e)*1.0/5 as last_five_err_opp,
    CASE
        WHEN opp.games_played > 0 THEN opp.wins*1.0/(opp.games_played)
        ELSE 0.0
    END as win_pct_opp,
	opp.streak as streak_opp,
    opp."lg rk" as opponent_rank,
    teams.win as won_game
 FROM teams
 JOIN teams as opp
 ON teams.opponent = opp.team AND teams.date = opp.date
 WHERE teams.team < opp.team
 ORDER by teams.date

"""
with sqlite3.connect('../team.db') as conn:
    df = pd.read_sql(query, conn)

train, test = train_test_split(df, test_size=0.2)
df.head() 

,year,date,team,opponent,last_five_run,last_five_hit,last_five_err,win_pct,streak,team_rank,last_five_run_opp,last_five_hit_opp,last_five_err_opp,win_pct_opp,streak_opp,opponent_rank,won_game
0,2016,2016-04-03,Kansas City Royals,New York Mets,0.0,0.0,0.0,0.0,0,2,0.0,0.0,0.0,0.0,0,1,1
1,2016,2016-04-03,Pittsburgh Pirates,St. Louis Cardinals,0.0,0.0,0.0,0.0,0,3,0.0,0.0,0.0,0.0,0,1,1
2,2016,2016-04-03,Tampa Bay Rays,Toronto Blue Jays,0.0,0.0,0.0,0.0,0,2,0.0,0.0,0.0,0.0,0,3,0
3,2016,2016-04-04,Arizona Diamondbacks,Colorado Rockies,0.0,0.0,0.0,0.0,0,4,0.0,0.0,0.0,0.0,0,8,0
4,2016,2016-04-04,Atlanta Braves,Washington Nationals,0.0,0.0,0.0,0.0,0,6,0.0,0.0,0.0,0.0,0,13,0


#### EDA
Let's look at the correlation between the variables to see if this is viable

In [3]:
df.drop(['team','date','opponent','year'],axis=1).corr()

,last_five_run,last_five_hit,last_five_err,win_pct,streak,team_rank,last_five_run_opp,last_five_hit_opp,last_five_err_opp,win_pct_opp,streak_opp,opponent_rank,won_game
last_five_run,1.000000,0.816452,0.121503,0.293152,0.307989,0.066753,0.153449,0.209074,0.120735,0.023497,-0.148543,-0.214026,0.020258
last_five_hit,0.816452,1.000000,0.183376,0.219612,0.167908,0.033190,0.232476,0.325734,0.137003,0.091523,-0.072297,-0.089873,-0.002269
last_five_err,0.121503,0.183376,1.000000,-0.013477,-0.091261,-0.024860,0.142616,0.143946,0.050146,0.069725,0.057978,0.060512,-0.016282
win_pct,0.293152,0.219612,-0.013477,1.000000,0.324935,0.126521,0.009420,0.070182,0.063040,-0.138080,-0.146824,-0.769160,0.084666
streak,0.307989,0.167908,-0.091261,0.324935,1.000000,0.149618,-0.167846,-0.098172,0.049082,-0.146394,-0.497383,-0.321046,0.049532
team_rank,0.066753,0.033190,-0.024860,0.126521,0.149618,1.000000,-0.203586,-0.091086,0.107176,-0.749543,-0.310727,-0.135005,0.205676
last_five_run_opp,0.153449,0.232476,0.142616,0.009420,-0.167846,-0.203586,1.000000,0.816685,0.089415,0.268677,0.317203,0.073725,-0.037035
last_five_hit_opp,0.209074,0.325734,0.143946,0.070182,-0.098172,-0.091086,0.816685,1.000000,0.168345,0.203118,0.194752,0.052021,-0.017158
last_five_err_opp,0.120735,0.137003,0.050146,0.063040,0.049082,0.107176,0.089415,0.168345,1.000000,-0.048122,-0.100338,-0.013502,0.017301
win_pct_opp,0.023497,0.091523,0.069725,-0.138080,-0.146394,-0.749543,0.268677,0.203118,-0.048122,1.000000,0.304340,0.114310,-0.063812


Looks like we have some good correlation with the ranking columns as well as the win percentages.  I think this looks promising as a first pass.

In [4]:
trainx = train.drop(['year','date','team','opponent','won_game'],axis=1)
testx = test.drop(['year','date','team','opponent','won_game'],axis=1)
testy = test['won_game']

Since we ultimately want to maximize the probability estimate being correct as opposed to the accuracy (more on this later), we will use negative log loss for our model.

In [5]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    logregCV = LogisticRegressionCV(cv=10,n_jobs=-1,scoring='neg_log_loss')
    logregCV.fit(trainx,train['won_game'])
logregCV.score(testx,test['won_game'])

C:\Users\Alekche\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\Alekche\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\Alekche\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\Alekche\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension

-0.6290393468353607

In [6]:
preds = logregCV.predict_proba(testx)
test['prediction'] = preds[:,1]
1 - np.sum(abs(test['won_game'] - np.round(test['prediction'])))/len(test)

C:\Users\Alekche\anaconda3\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\Alekche\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\Alekche\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


0.6568527918781726

Our model is able to get the game outcome correct ~66% of the time.  Not bad for a first pass.

# The quest for cash
In order to make money betting, what we really need to do is predict odds more accurately than the betting line.  Essentially, we can profit off of scenarios where the payout of an event is dispropotionally higher than the actual odds of the event occurring.

Example: team A has posted odds of +200 to win.  This equates to 2:1 odds, or a 33% chance of the event occurring.  The expected value from the posted odds for a 100 dollar bet is
$$EV_{book} = -100*0.66 + 200*0.33 = 0$$
If we think the true chances of team A winning is actually closer to 50% then the expected value (for a 100 dollar bet) is:
$$EV: -100*0.5 + 200*0.5 = 100$$
And we will make money over the long run if we repeatedly bet like this.  In practice this is very difficult; a bookmaker typically sets the odds such that they are not even; that is if you calculated the probability of either event happening you'd get a value over 100% (This is how a bookmaker guarantees they make money).

Example:
Game 1 of the 2016 season has Pittsburgh paying out -119 and the Cardinals paying out +109.  This translates to probabilities of 54.3 and 47.8 respectively leading to a total 102.1%!  Thus; here we have negative expected value from betting (bet 119 dollars to win 100):
$$EV_{Pittsburgh} = 100*0.543 - 119*0.478 = -2.58 $$

American odds are a little goofy - we will be using probability to compute our model's betting performance.

## Model1 performance
To evaluate the logistic regression model's performance, we will see where there are discrepancies between the output win/loss probability and place a bet which maximizes our EV.  We will assume we are betting right before the game and use closing odds.

In [7]:
# SQL Query for this data
query = """
SELECT 
	*
FROM odds
ORDER BY date
"""
with sqlite3.connect('../team.db') as conn:
    odds_df = pd.read_sql(query, conn)
odds_df.head()

,team,date,open,close,run_line,run_odds,ou_open,ou_open_odds,ou_close,ou_close_odds
0,St. Louis Cardinals,2016-04-03,-115,109.0,1.5,-230.0,6.5,-105,6.5,-110
1,Pittsburgh Pirates,2016-04-03,-105,-119.0,-1.5,190.0,6.5,-115,6.5,-110
2,Toronto Blue Jays,2016-04-03,-115,111.0,1.5,-210.0,7,-115,7.0,-130
3,Tampa Bay Rays,2016-04-03,-105,-121.0,-1.5,180.0,7,-105,7.0,110
4,New York Mets,2016-04-03,-119,-120.0,-1.5,145.0,7,-120,8.0,-103


In [8]:
test_ev = test.merge(odds_df, on=['team','date'])
test_ev = test_ev.merge(odds_df[['team','date','close']],left_on=['opponent','date'],right_on=['team','date'], suffixes=('','_opponent'))

In [9]:
def odds_to_prob(value):
    if value <0:
        return value/(value-100)
    else:
        return 100/value


def odds_to_payout(value):
    if value >0:
        return value
    else:
        return 100*abs(100/value)


def calc_ev(row, debug=False):
    """Uses closing odds to calculate result"""
    if row['close'] is None:
        return 0, 0
    #print(f"res: {row['won_game']} pred: {row['prediction']}, team prob: {odds_to_prob(row['close'])}, opp prob: {odds_to_prob(row['close_opponent'])}")
    pay_team = odds_to_payout(row['close'])
    pay_opp = odds_to_payout(row['close_opponent'])


    ev_team = row['prediction']*pay_team - (1-row['prediction'])*100
    ev_opp = (1-row['prediction'])*pay_opp - row['prediction']*100

    if debug:
        print(f"team: {row['close']} payout: {pay_team}")
        print(f"opp: {row['close_opponent']} payout: {pay_opp}")
        print(f"pred:{row['prediction']}, ev_team: {ev_team}, ev_opp: {ev_opp}")

    return ev_team, ev_opp


def gamble(row):
    """Result of gambling $100 using EV as a strategy"""
    ev_team, ev_opp = calc_ev(row)

    if ev_team > ev_opp and row['won_game']:
        #We bet on the team and win
        return odds_to_payout(row['close'])
    elif ev_opp > ev_team and not row['won_game']:
        #We bet on the opponent and win
        return odds_to_payout(row['close_opponent'])
    else:
        #We lose $100
        return -100
        

In [10]:
test_ev['gamble'] = test_ev.apply(gamble,axis=1)
np.mean(test_ev['gamble'])

20.515643204041396

Not bad!  Seems that our model on average gets us $20 per bet in profit over the long run.  Let's see if there is a difference by year.

In [11]:
test_ev.groupby('year')['gamble'].mean()

year
2016    15.581993
2017    20.772306
2018    20.326911
2019    19.957466
2020    34.895980
2021    20.719084
Name: gamble, dtype: float64

Interesting - seems that things are pretty stable aside from 2020 where we have an amazing year.  I did the year by year comparison because I suspect that as of today (2024), the popularity of AI/ML has made the betting lines substantially more competitive.  I would need more data to be sure.

It is very odd that the 2020 season - which was shortened to just 60 games due to covid - has such outstanding performance from our model.  My hypothesis is that it is not the model doing well, but rather the betting odds that year being off, which is the contributing factor.

In [12]:
def test_model(predictions, test_df, odds_df):
    """Perform all steps above on a new model"""
    test_df['prediction'] = predictions
    
    test_ev = test_df.merge(odds_df, on=['team','date'])
    test_ev = test_ev.merge(odds_df[['team','date','close']],left_on=['opponent','date'],right_on=['team','date'], suffixes=('','_opponent'))
    test_ev['gamble'] = test_ev.apply(gamble,axis=1)
    return test_ev

# A more complex model
Let's see what sort of performance gain we can get if we try to use some more complex models in our predictions.  We'll try tree-based models first.

In [13]:
import xgboost as xgb

xgtrain = xgb.DMatrix(trainx, train['won_game'])
xgtest = xgb.DMatrix(testx, testy)

In [14]:
params = {"objective": "binary:logistic"}
eval = [(xgtrain, "train"), (xgtest, "validation")]
n = 40
model = xgb.train(
    params=params,
    dtrain=xgtrain,
    num_boost_round=n,
    evals=eval
)

[0]	train-logloss:0.65935	validation-logloss:0.66402
[1]	train-logloss:0.63545	validation-logloss:0.64411
[2]	train-logloss:0.61761	validation-logloss:0.63057
[3]	train-logloss:0.60489	validation-logloss:0.62239
[4]	train-logloss:0.59152	validation-logloss:0.61389
[5]	train-logloss:0.58186	validation-logloss:0.60823
[6]	train-logloss:0.57348	validation-logloss:0.60497
[7]	train-logloss:0.56830	validation-logloss:0.60167
[8]	train-logloss:0.56176	validation-logloss:0.59960
[9]	train-logloss:0.55638	validation-logloss:0.59852
[10]	train-logloss:0.55058	validation-logloss:0.59678
[11]	train-logloss:0.54658	validation-logloss:0.59623
[12]	train-logloss:0.54287	validation-logloss:0.59605
[13]	train-logloss:0.53830	validation-logloss:0.59548
[14]	train-logloss:0.53456	validation-logloss:0.59490
[15]	train-logloss:0.53089	validation-logloss:0.59331
[16]	train-logloss:0.52638	validation-logloss:0.59407
[17]	train-logloss:0.52159	validation-logloss:0.59288
[18]	train-logloss:0.52014	validation-

In [15]:
preds = model.predict(xgtest)
test['prediction'] = preds
1 - np.sum(abs(test['won_game'] - np.round(test['prediction'])))/len(test)

0.6741116751269036

In [18]:
np.mean(test_model(preds, test, odds_df)['gamble'])

26.43565487341076

Wow - XGboost does quite a bit better.  Let's see the yearly breakdown.

In [19]:
test_model(preds, test, odds_df).groupby('year')['gamble'].mean()

year
2016    26.002756
2017    27.798343
2018    17.264256
2019    29.036795
2020    42.069736
2021    25.980791
Name: gamble, dtype: float64